In [26]:
from typing import Callable 
from numpy.typing import ArrayLike
from random import random
from scipy.stats import expon, kstest
import numpy as np
from math import log

---
# Ejercicio 9
En un estudio de vibraciones, una muestra aleatoria de 15 componentes del avión fueron
sometidos a fuertes vibraciones hasta que se evidenciaron fallas estructurales. Los datos proporcionados son los minutos transcurridos hasta que se evidenciaron dichas fallas.
$$
1.6, \quad 10.3, \quad 3.5, \quad 13.5, \quad 18.4, \quad 7.7, \quad 24.3, \quad 10.7, \quad 8.4, \quad 4.9, \quad 7.9, \quad 12, \quad 16.2, \quad 6.8, \quad 14.7
$$



Pruebe la hipótesis nula de que estas observaciones pueden ser consideradas como una muestra de la
distribución exponencial.

Como desconocemos el parámetro $\lambda$ de la distribución exponencial debemos estimarlo. Luego aplicamos el método de forma normal.

Tenemos que si X es una v.a con distribución exponencial de parámetro $\lambda$ desconocido su esperanza viene dada por:
$$
E[X] = \frac{1}{\lambda}
$$ 

Como queremos estimar $\lambda$:
$$
\lambda = \frac{1}{E[X]}
$$

Luego utilizando que la media muestral es un estimador de $E[X]$ obtenemos que un estimador para $\hat{\lambda}$ es:
$$
\hat{\lambda} = \frac{1}{\bar{X}(n)} = \frac{1}{\sum_{i=0}^n x_i}
$$

# MÉTODO DEL GORDO KOLMOGOROV SMIRNOV 🥴

In [23]:
SAMPLES = [
    1.6, 10.3, 3.5,
    13.5, 18.4, 7.7,
    24.3, 10.7, 8.4,
    4.9, 7.9, 12,
    16.2, 6.8, 14.7
]

NSAMPLES = len(SAMPLES)

#Estimación del parámetro
LAMBDA_HAT = NSAMPLES / sum(SAMPLES)

def NEXP(lamda:float) -> float:
    U = 1 - random()
    return -log(U) / lamda

In [24]:
#Definimos el estadístico
def KS_statistic(Nsamples:int, samples:ArrayLike, G:Callable[[ArrayLike], ArrayLike]) -> float:
    Fe_samples = np.arange(1, Nsamples + 1, 1) / Nsamples
    G_values = G(samples)

    Fe_minus = np.arange(0, Nsamples) / Nsamples
    Fe_plus = np.arange(1, Nsamples + 1) / Nsamples

    D_minus = G_values - Fe_minus
    D_plus = Fe_plus - G_values
    
    d = np.max(np.concatenate([D_plus, D_minus]))
    return d

#Test de Kolomogorov-Smirnov
def KS_test(Nsim:int, Nsamples:int,
            samples:ArrayLike, alpha:float,
            G:Callable[[ArrayLike], ArrayLike]):

    p_value = 0
    #Ordeño muestras 🐄
    samples = np.sort(samples)
    #Calculo estadístico inicial
    d0 = KS_statistic(Nsamples=Nsamples, samples=samples, G=G)

    print(f"🧐 D estadístico:{round(d0, 4)}")

    for _ in range(Nsim):
        #Generamos muestras aleatorias para estimar el p_valor
        uniform_samples = np.sort([NEXP(lamda=LAMBDA_HAT) for _ in range(Nsamples)])
        d_sim = KS_statistic(Nsamples=Nsamples, samples=uniform_samples, G=G)
        if d_sim >= d0: 
            p_value += 1
        
    p_value = p_value / Nsim
    print(f"☝️ p-valor obtenido: {round(p_value, 4)}")

    if p_value > alpha:
        print(f"😲☝️ Como {p_value} > {alpha}:")
        print("\t 😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"😲☝️ Como {p_value} <= {alpha}:")
        print(f"\t 🔴 Se rechaza Ho con una confianza del {int(100 * (1-alpha))}%")

In [25]:
ALPHA = 0.05
KS_test(Nsim=10_000, Nsamples=NSAMPLES, samples=SAMPLES, alpha=ALPHA, G=expon(scale=1/LAMBDA_HAT).cdf)

🧐 D estadístico:0.2695
☝️ p-valor obtenido: 0.1914
😲☝️ Como 0.1914 > 0.05:
	 😒 No hay evidencia suficiente para rechazar Ho


# Implementación chiteada de SCIPY 🐍

In [31]:
# Aplicamos el test de Kolmogorov-Smirnov contra la exponencial con ese scale
D, p_value = kstest(SAMPLES, 'expon', args=(0, 1/LAMBDA_HAT))

print(f"📈 D estadístico: {round(D, 4)}")
print(f"📊 p-valor: {round(p_value, 4)}")

📈 D estadístico: 0.2695
📊 p-valor: 0.188
